#성공 데이터 전처리
4가지 전처리 과정이 포함됩니다.

resent18<br>
resent18 + robotdata<br>
resnet18 -> pca<br>
resnet18 -> pca + robotdata<br>

In [1]:
"""
apply resent18 (for success)

structure
data = {
    "success_episode{episode_number}_steps{episode_length//5+1}":
        [episode_length//5+1, 512*3] #front, top, wrist
    }
"""
import os
import re
import pickle
from typing import Dict, Tuple, List

import numpy as np
import torch
import torch.nn as nn
from PIL import Image
from torchvision import models, transforms
from torchvision.models import ResNet18_Weights
os.makedirs("dataset", exist_ok=True)

# ====== 설정 ======
ROOT_DIR = "/AILAB-summer-school-2025/success_data_raw"   # 입력 루트
OUT_PATH = "dataset/success_data_resnet18_robotX.pkl"                    # 출력 파일
VIEWS = ("front", "top", "wrist")                         # 처리할 뷰

# 이미지 전처리 (요청 그대로)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std= [0.229,0.224,0.225]),
])

# 에피소드 폴더명: success_episode{num}_steps{episode_length}
EP_DIR_RE = re.compile(r"^success_episode(\d+)_steps(\d+)$")


def parse_episode_dirname(name: str) -> Tuple[int, int]:
    m = EP_DIR_RE.match(name)
    if not m:
        raise ValueError(f"Invalid episode dir name: {name}")
    return int(m.group(1)), int(m.group(2))


def list_episode_dirs(root: str) -> List[Tuple[int, int, str, str]]:
    """(ep_num, ep_len, abs_path, dir_name) 를 ep_num 오름차순으로 반환"""
    eps = []
    for name in os.listdir(root):
        p = os.path.join(root, name)
        if not os.path.isdir(p):
            continue
        if EP_DIR_RE.match(name):
            ep_num, ep_len = parse_episode_dirname(name)
            eps.append((ep_num, ep_len, p, name))
    eps.sort(key=lambda x: x[0])
    return eps


def expected_steps(ep_len: int) -> List[int]:
    """0부터 (ep_len-1)까지 5 간격 스텝 (예: ep_len=320 -> [0,5,...,315], 총 64개)"""
    return list(range(0, ep_len, 5))


def img_path(ep_path: str, view: str, step: int) -> str:
    """뷰/스텝에 해당하는 이미지 경로"""
    return os.path.join(ep_path, f"{view}_view", f"{view}_view_{step}.png")


@torch.inference_mode()
def extract_feature(model: nn.Module, path: str, device: torch.device) -> np.ndarray:
    with Image.open(path).convert("RGB") as img:
        x = transform(img).unsqueeze(0).to(device)
        f = model(x).squeeze(0).detach().cpu().numpy().astype(np.float32)  # (512,)
    return f


def main():
    # 모델 준비 (ResNet18 + ImageNet 가중치, FC 제거 → 512차원)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
    model.fc = nn.Identity()
    model.eval().to(device)

    # 에피소드 스캔
    episodes = list_episode_dirs(ROOT_DIR)
    if not episodes:
        raise RuntimeError(f"No valid episode directories found under '{ROOT_DIR}'")

    data: Dict[str, np.ndarray] = {}

    for ep_num, ep_len, ep_path, ep_name in episodes:
        steps = expected_steps(ep_len)                 # 길이 = ep_len // 5
        print(f"[Episode {ep_num:>4}] {ep_name} -> steps expected: {len(steps)} (0..{ep_len-1} by 5)")

        # 파일 존재 검증(세 뷰 모두)
        missing = []
        for s in steps:
            for v in VIEWS:
                p = img_path(ep_path, v, s)
                if not os.path.isfile(p):
                    missing.append(p)
        if missing:
            # 어떤 에피소드에서든 필수 스텝이 비어있으면 바로 알려주고 중단
            preview = "\n".join(missing[:20])
            tail = "" if len(missing) <= 20 else f"\n... (+{len(missing)-20} more)"
            raise FileNotFoundError(
                f"[{ep_name}] Missing required image files:\n{preview}{tail}"
            )

        # 특징 추출
        episode_feats = []
        for s in steps:
            paths = [img_path(ep_path, v, s) for v in VIEWS]
            feats = [extract_feature(model, p, device) for p in paths]  # 3 x (512,)
            merged = np.concatenate(feats, axis=0)  # (1536,)
            episode_feats.append(merged)
            #print(f"    - step {s:>4}  OK")

        arr = np.stack(episode_feats, axis=0)  # (T, 1536), T = ep_len // 5
        key = f"success_episode{ep_num}_steps{len(steps)}"
        data[key] = arr
        print(f"  -> Added episode: key='{key}', array shape={arr.shape}")

    # 저장
    with open(OUT_PATH, "wb") as f:
        pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

    print(f"\nSaved: {OUT_PATH}")
    print(f"Total episodes: {len(data)}")


if __name__ == "__main__":
    main()


[Episode    0] success_episode0_steps321 -> steps expected: 65 (0..320 by 5)
  -> Added episode: key='success_episode0_steps65', array shape=(65, 1536)
[Episode    1] success_episode1_steps341 -> steps expected: 69 (0..340 by 5)
  -> Added episode: key='success_episode1_steps69', array shape=(69, 1536)
[Episode    2] success_episode2_steps314 -> steps expected: 63 (0..313 by 5)
  -> Added episode: key='success_episode2_steps63', array shape=(63, 1536)
[Episode    3] success_episode3_steps302 -> steps expected: 61 (0..301 by 5)
  -> Added episode: key='success_episode3_steps61', array shape=(61, 1536)
[Episode    4] success_episode4_steps313 -> steps expected: 63 (0..312 by 5)
  -> Added episode: key='success_episode4_steps63', array shape=(63, 1536)
[Episode    5] success_episode5_steps317 -> steps expected: 64 (0..316 by 5)
  -> Added episode: key='success_episode5_steps64', array shape=(64, 1536)
[Episode    6] success_episode6_steps310 -> steps expected: 62 (0..309 by 5)
  -> Added 

In [2]:
"""
apply resent18 + robotdata (for success)

structure
data = {
    "success_episode{episode_number}_steps{episode_length//5+1}":
        [episode_length//5+1, 512*3+7] #front, top, wrist, robot_data
    }
"""
import os
import re
import pickle
from typing import Dict, Tuple, List, Callable

import numpy as np
import torch
import torch.nn as nn
from PIL import Image
from torchvision import models, transforms
from torchvision.models import ResNet18_Weights

# ====== 설정 ======
ROOT_DIR = "/AILAB-summer-school-2025/success_data_raw"   # 입력 루트
OUT_PATH = "dataset/success_data_resnet18_robotO.pkl"                    # 출력 파일
VIEWS = ("front", "top", "wrist")                         # 처리할 뷰

# 이미지 전처리 (요청 그대로)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std= [0.229,0.224,0.225]),
])

# 에피소드 폴더명: success_episode{num}_steps{episode_length}
EP_DIR_RE = re.compile(r"^success_episode(\d+)_steps(\d+)$")


def parse_episode_dirname(name: str) -> Tuple[int, int]:
    m = EP_DIR_RE.match(name)
    if not m:
        raise ValueError(f"Invalid episode dir name: {name}")
    return int(m.group(1)), int(m.group(2))


def list_episode_dirs(root: str) -> List[Tuple[int, int, str, str]]:
    """(ep_num, ep_len, abs_path, dir_name) 를 ep_num 오름차순으로 반환"""
    eps = []
    for name in os.listdir(root):
        p = os.path.join(root, name)
        if not os.path.isdir(p):
            continue
        if EP_DIR_RE.match(name):
            ep_num, ep_len = parse_episode_dirname(name)
            eps.append((ep_num, ep_len, p, name))
    eps.sort(key=lambda x: x[0])
    return eps


def expected_steps(ep_len: int) -> List[int]:
    """0부터 (ep_len-1)까지 5 간격 스텝 (예: ep_len=320 -> [0,5,...,315], 총 64개)"""
    return list(range(0, ep_len, 5))


def img_path(ep_path: str, view: str, step: int) -> str:
    """뷰/스텝에 해당하는 이미지 경로"""
    return os.path.join(ep_path, f"{view}_view", f"{view}_view_{step}.png")


def coerce_2d_float(arr, dim2: int) -> np.ndarray:
    """리스트/obj-array도 (T,dim2) float32로 강제."""
    if isinstance(arr, np.ndarray) and arr.dtype != object:
        out = arr
    else:
        out = np.asarray([np.asarray(x).reshape(-1) for x in arr], dtype=np.float32)
    if out.ndim != 2 or out.shape[1] != dim2:
        raise ValueError(f"Expected shape (T,{dim2}) but got {out.shape}")
    return out.astype(np.float32, copy=False)


@torch.inference_mode()
def extract_feature(model: nn.Module, path: str, device: torch.device) -> np.ndarray:
    with Image.open(path).convert("RGB") as img:
        x = transform(img).unsqueeze(0).to(device)
        f = model(x).squeeze(0).detach().cpu().numpy().astype(np.float32)  # (512,)
    return f


def pick_pose_indexer(ep_len: int, steps: List[int], T_pose: int) -> Callable[[int, int], int]:
    """
    EE_pose 인덱싱 전략을 선택해 indexer(i, s)를 반환.
    - 가장 흔한 케이스: T_pose == len(steps)  -> i로 인덱싱
    - 프레임 전부 저장: T_pose >= ep_len      -> s로 인덱싱
    - 포함 케이스: T_pose == len(steps)+1     -> i로 인덱싱(마지막 한 칸 여유)
    - 다른 경우는 모호 → 에러
    """
    T_steps = len(steps)
    if T_pose == T_steps or T_pose == T_steps + 1:
        return lambda i, s: i
    if T_pose >= ep_len:
        return lambda i, s: s
    # 보수적으로 한 번 더 허용: ep_len//5 == T_pose (동일 의미)
    if T_pose == (ep_len // 5):
        return lambda i, s: i
    raise ValueError(
        f"Cannot map EE_pose length {T_pose} to steps={T_steps} (episode_length={ep_len}). "
        f"Expected one of {{T_steps, T_steps+1, >= ep_len}}."
    )


def main():
    # 모델 준비 (ResNet18 + ImageNet 가중치, FC 제거 → 512차원)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
    model.fc = nn.Identity()
    model.eval().to(device)

    # 에피소드 스캔
    episodes = list_episode_dirs(ROOT_DIR)
    if not episodes:
        raise RuntimeError(f"No valid episode directories found under '{ROOT_DIR}'")

    data: Dict[str, np.ndarray] = {}

    for ep_num, ep_len, ep_path, ep_name in episodes:
        steps = expected_steps(ep_len)  # 길이 = ep_len // 5
        print(f"[Episode {ep_num:>4}] {ep_name} -> steps expected: {len(steps)} (0..{ep_len-1} by 5)")

        # robot_state.npz 로드 및 EE_pose 확보
        state_path = os.path.join(ep_path, "robot_state.npz")
        if not os.path.isfile(state_path):
            raise FileNotFoundError(f"[{ep_name}] robot_state.npz not found: {state_path}")
        state = np.load(state_path, allow_pickle=True)
        if "EE_pose" not in state:
            raise KeyError(f"[{ep_name}] 'EE_pose' key not found in {state_path}")
        ee_pose = coerce_2d_float(state["EE_pose"], dim2=7)  # (T_pose, 7)
        T_pose = ee_pose.shape[0]

        # 이미지 파일 존재 검증(세 뷰 모두)
        missing = []
        for s in steps:
            for v in VIEWS:
                p = img_path(ep_path, v, s)
                if not os.path.isfile(p):
                    missing.append(p)
        if missing:
            preview = "\n".join(missing[:20])
            tail = "" if len(missing) <= 20 else f"\n... (+{len(missing)-20} more)"
            raise FileNotFoundError(
                f"[{ep_name}] Missing required image files:\n{preview}{tail}"
            )

        # EE_pose 인덱싱 모드 선택
        indexer = pick_pose_indexer(ep_len, steps, T_pose)

        # 특징 추출 + EE_pose concat
        episode_feats = []
        for i, s in enumerate(steps):
            paths = [img_path(ep_path, v, s) for v in VIEWS]
            feats = [extract_feature(model, p, device) for p in paths]  # 3 x (512,)
            pose = ee_pose[indexer(i, s)].astype(np.float32, copy=False) # (7,)
            merged = np.concatenate(feats + [pose], axis=0)              # (1536 + 7 = 1543,)
            episode_feats.append(merged)
            #print(f"    - step {s:>4}  OK")

        arr = np.stack(episode_feats, axis=0)  # (T, 1543), T = ep_len // 5
        key = f"success_episode{ep_num}_steps{len(steps)}"
        data[key] = arr
        print(f"  -> Added episode: key='{key}', array shape={arr.shape}")

    # 저장
    with open(OUT_PATH, "wb") as f:
        pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

    print(f"\nSaved: {OUT_PATH}")
    print(f"Total episodes: {len(data)}")


if __name__ == "__main__":
    main()


[Episode    0] success_episode0_steps321 -> steps expected: 65 (0..320 by 5)
  -> Added episode: key='success_episode0_steps65', array shape=(65, 1543)
[Episode    1] success_episode1_steps341 -> steps expected: 69 (0..340 by 5)
  -> Added episode: key='success_episode1_steps69', array shape=(69, 1543)
[Episode    2] success_episode2_steps314 -> steps expected: 63 (0..313 by 5)
  -> Added episode: key='success_episode2_steps63', array shape=(63, 1543)
[Episode    3] success_episode3_steps302 -> steps expected: 61 (0..301 by 5)
  -> Added episode: key='success_episode3_steps61', array shape=(61, 1543)
[Episode    4] success_episode4_steps313 -> steps expected: 63 (0..312 by 5)
  -> Added episode: key='success_episode4_steps63', array shape=(63, 1543)
[Episode    5] success_episode5_steps317 -> steps expected: 64 (0..316 by 5)
  -> Added episode: key='success_episode5_steps64', array shape=(64, 1543)
[Episode    6] success_episode6_steps310 -> steps expected: 62 (0..309 by 5)
  -> Added 

In [3]:
"""
apply resnet18 -> pca (for success)

structure
data = {
    "success_episode{episode_number}_steps{episode_length//5+1}":
        [episode_length//5+1, 64*3] #front, top, wrist
    }
"""

import os
import re
import pickle
from typing import Dict, Tuple, List

import numpy as np
import torch
import torch.nn as nn
from PIL import Image
from torchvision import models, transforms
from torchvision.models import ResNet18_Weights

# ====== 설정 ======
ROOT_DIR = "/AILAB-summer-school-2025/success_data_raw"   # 입력 루트
OUT_PATH = "dataset/success_data_resnet18_pca_robotX.pkl"                # 출력 파일 (PCA 버전으로 구분)
VIEWS = ("front", "top", "wrist")                         # 처리할 뷰

# 이미지 전처리 (요청 그대로)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std= [0.229,0.224,0.225]),
])

# 에피소드 폴더명: success_episode{num}_steps{episode_length}
EP_DIR_RE = re.compile(r"^success_episode(\d+)_steps(\d+)$")

def parse_episode_dirname(name: str) -> Tuple[int, int]:
    m = EP_DIR_RE.match(name)
    if not m:
        raise ValueError(f"Invalid episode dir name: {name}")
    return int(m.group(1)), int(m.group(2))

def list_episode_dirs(root: str) -> List[Tuple[int, int, str, str]]:
    """(ep_num, ep_len, abs_path, dir_name) 를 ep_num 오름차순으로 반환"""
    eps = []
    for name in os.listdir(root):
        p = os.path.join(root, name)
        if not os.path.isdir(p):
            continue
        if EP_DIR_RE.match(name):
            ep_num, ep_len = parse_episode_dirname(name)
            eps.append((ep_num, ep_len, p, name))
    eps.sort(key=lambda x: x[0])
    return eps

def expected_steps(ep_len: int) -> List[int]:
    """0부터 (ep_len-1)까지 5 간격 스텝 (예: ep_len=320 -> [0,5,...,315], 총 64개)"""
    return list(range(0, ep_len, 5))

def img_path(ep_path: str, view: str, step: int) -> str:
    return os.path.join(ep_path, f"{view}_view", f"{view}_view_{step}.png")

def get_sorted_image_paths(ep_path: str, view: str, steps: List[int]) -> List[str]:
    """요구 스텝 순서대로 이미지 경로 생성 & 존재 확인"""
    paths = []
    missing = []
    for s in steps:
        p = img_path(ep_path, view, s)
        if os.path.isfile(p):
            paths.append(p)
        else:
            missing.append(p)
    if missing:
        preview = "\n".join(missing[:20])
        tail = "" if len(missing) <= 20 else f"\n... (+{len(missing)-20} more)"
        raise FileNotFoundError(f"[{os.path.basename(ep_path)}] Missing {view} images:\n{preview}{tail}")
    return paths

def load_pca_model(view: str):
    pca_path = f"model/model_pca_{view}_view.pkl"
    if not os.path.isfile(pca_path):
        raise FileNotFoundError(f"PCA model not found: {pca_path}")
    with open(pca_path, "rb") as f:
        return pickle.load(f)

@torch.inference_mode()
def extract_resnet_feature(model: nn.Module, img_path: str, device: torch.device) -> np.ndarray:
    with Image.open(img_path).convert("RGB") as img:
        x = transform(img).unsqueeze(0).to(device)
        f = model(x).squeeze(0).detach().cpu().numpy().astype(np.float32)  # (512,)
    return f

def main():
    # 모델 준비 (ResNet18 + ImageNet 가중치, FC 제거 → 512차원)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
    model.fc = nn.Identity()
    model.eval().to(device)

    # PCA 모델 미리 로드
    pca_models: Dict[str, object] = {v: load_pca_model(v) for v in VIEWS}

    # 에피소드 스캔
    episodes = list_episode_dirs(ROOT_DIR)
    if not episodes:
        raise RuntimeError(f"No valid episode directories found under '{ROOT_DIR}'")

    data: Dict[str, np.ndarray] = {}

    for ep_num, ep_len, ep_path, ep_name in episodes:
        steps = expected_steps(ep_len)  # 길이 = ep_len // 5
        print(f"[Episode {ep_num:>4}] {ep_name} -> steps expected: {len(steps)} (0..{ep_len-1} by 5)")

        # 뷰별 이미지 리스트 수집(존재 검증 포함)
        img_lists: Dict[str, List[str]] = {}
        for v in VIEWS:
            img_lists[v] = get_sorted_image_paths(ep_path, v, steps)

        # 뷰별 ResNet(512) → PCA(64)
        latents_64 = []
        for v in VIEWS:
            paths = img_lists[v]
            feats512 = []
            for p in paths:
                f512 = extract_resnet_feature(model, p, device)  # (512,)
                feats512.append(f512)
            feats512 = np.stack(feats512, axis=0)               # (T,512)
            feats64  = pca_models[v].transform(feats512).astype(np.float32, copy=False)  # (T,64)
            latents_64.append(feats64)

        # 세 뷰 concat → (T, 64*3 = 192)
        arr = np.concatenate(latents_64, axis=1)  # (T, 192)
        key = f"success_episode{ep_num}_steps{len(steps)}"
        data[key] = arr
        print(f"  -> Added episode: key='{key}', array shape={arr.shape}")

    # 저장
    with open(OUT_PATH, "wb") as f:
        pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

    print(f"\nSaved: {OUT_PATH}")
    print(f"Total episodes: {len(data)}")

if __name__ == "__main__":
    main()


[Episode    0] success_episode0_steps321 -> steps expected: 65 (0..320 by 5)
  -> Added episode: key='success_episode0_steps65', array shape=(65, 192)
[Episode    1] success_episode1_steps341 -> steps expected: 69 (0..340 by 5)
  -> Added episode: key='success_episode1_steps69', array shape=(69, 192)
[Episode    2] success_episode2_steps314 -> steps expected: 63 (0..313 by 5)
  -> Added episode: key='success_episode2_steps63', array shape=(63, 192)
[Episode    3] success_episode3_steps302 -> steps expected: 61 (0..301 by 5)
  -> Added episode: key='success_episode3_steps61', array shape=(61, 192)
[Episode    4] success_episode4_steps313 -> steps expected: 63 (0..312 by 5)
  -> Added episode: key='success_episode4_steps63', array shape=(63, 192)
[Episode    5] success_episode5_steps317 -> steps expected: 64 (0..316 by 5)
  -> Added episode: key='success_episode5_steps64', array shape=(64, 192)
[Episode    6] success_episode6_steps310 -> steps expected: 62 (0..309 by 5)
  -> Added episod

In [4]:
"""
apply resnet18 -> pca + robotdata (for success)

structure
data = {
    "success_episode{episode_number}_steps{episode_length//5+1}":
        [episode_length//5+1, 64*3+7] #front, top, wrist, robot_data
    }
"""


import os
import re
import pickle
from typing import Dict, Tuple, List

import numpy as np
import torch
import torch.nn as nn
from PIL import Image
from torchvision import models, transforms
from torchvision.models import ResNet18_Weights

# ====== 설정 ======
ROOT_DIR = "/AILAB-summer-school-2025/success_data_raw"   # 입력 루트
OUT_PATH = "dataset/success_data_resnet18_pca_robotO.pkl"                # 출력 파일 (PCA 버전으로 구분)
VIEWS = ("front", "top", "wrist")                         # 처리할 뷰

# 이미지 전처리 (요청 그대로)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std= [0.229,0.224,0.225]),
])

# 에피소드 폴더명: success_episode{num}_steps{episode_length}
EP_DIR_RE = re.compile(r"^success_episode(\d+)_steps(\d+)$")

def parse_episode_dirname(name: str) -> Tuple[int, int]:
    m = EP_DIR_RE.match(name)
    if not m:
        raise ValueError(f"Invalid episode dir name: {name}")
    return int(m.group(1)), int(m.group(2))

def list_episode_dirs(root: str) -> List[Tuple[int, int, str, str]]:
    """(ep_num, ep_len, abs_path, dir_name) 를 ep_num 오름차순으로 반환"""
    eps = []
    for name in os.listdir(root):
        p = os.path.join(root, name)
        if not os.path.isdir(p):
            continue
        if EP_DIR_RE.match(name):
            ep_num, ep_len = parse_episode_dirname(name)
            eps.append((ep_num, ep_len, p, name))
    eps.sort(key=lambda x: x[0])
    return eps

def expected_steps(ep_len: int) -> List[int]:
    """0부터 (ep_len-1)까지 5 간격 스텝 (예: ep_len=320 -> [0,5,...,315], 총 64개)"""
    return list(range(0, ep_len, 5))

def img_path(ep_path: str, view: str, step: int) -> str:
    return os.path.join(ep_path, f"{view}_view", f"{view}_view_{step}.png")

def get_sorted_image_paths(ep_path: str, view: str, steps: List[int]) -> List[str]:
    """요구 스텝 순서대로 이미지 경로 생성 & 존재 확인"""
    paths = []
    missing = []
    for s in steps:
        p = img_path(ep_path, view, s)
        if os.path.isfile(p):
            paths.append(p)
        else:
            missing.append(p)
    if missing:
        preview = "\n".join(missing[:20])
        tail = "" if len(missing) <= 20 else f"\n... (+{len(missing)-20} more)"
        raise FileNotFoundError(f"[{os.path.basename(ep_path)}] Missing {view} images:\n{preview}{tail}")
    return paths

def load_pca_model(view: str):
    pca_path = f"model/model_pca_{view}_view.pkl"
    if not os.path.isfile(pca_path):
        raise FileNotFoundError(f"PCA model not found: {pca_path}")
    with open(pca_path, "rb") as f:
        return pickle.load(f)

@torch.inference_mode()
def extract_resnet_feature(model: nn.Module, img_path: str, device: torch.device) -> np.ndarray:
    with Image.open(img_path).convert("RGB") as img:
        x = transform(img).unsqueeze(0).to(device)
        f = model(x).squeeze(0).detach().cpu().numpy().astype(np.float32)  # (512,)
    return f

def main():
    # 모델 준비 (ResNet18 + ImageNet 가중치, FC 제거 → 512차원)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
    model.fc = nn.Identity()
    model.eval().to(device)

    # PCA 모델 미리 로드
    pca_models: Dict[str, object] = {v: load_pca_model(v) for v in VIEWS}

    # 에피소드 스캔
    episodes = list_episode_dirs(ROOT_DIR)
    if not episodes:
        raise RuntimeError(f"No valid episode directories found under '{ROOT_DIR}'")

    data: Dict[str, np.ndarray] = {}

    for ep_num, ep_len, ep_path, ep_name in episodes:
        steps = expected_steps(ep_len)  # 길이 = ep_len // 5
        print(f"[Episode {ep_num:>4}] {ep_name} -> steps expected: {len(steps)} (0..{ep_len-1} by 5)")

        # 뷰별 이미지 리스트 수집(존재 검증 포함)
        img_lists: Dict[str, List[str]] = {}
        for v in VIEWS:
            img_lists[v] = get_sorted_image_paths(ep_path, v, steps)

        # 뷰별 ResNet(512) → PCA(64)
        latents_64 = []
        for v in VIEWS:
            paths = img_lists[v]
            feats512 = []
            for p in paths:
                f512 = extract_resnet_feature(model, p, device)  # (512,)
                feats512.append(f512)
            feats512 = np.stack(feats512, axis=0)               # (T,512)
            feats64  = pca_models[v].transform(feats512).astype(np.float32, copy=False)  # (T,64)
            latents_64.append(feats64)

        # 세 뷰 concat → (T, 64*3 = 192)
        arr = np.concatenate(latents_64, axis=1)  # (T, 192)
        key = f"success_episode{ep_num}_steps{len(steps)}"
        data[key] = arr
        print(f"  -> Added episode: key='{key}', array shape={arr.shape}")

    # 저장
    with open(OUT_PATH, "wb") as f:
        pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

    print(f"\nSaved: {OUT_PATH}")
    print(f"Total episodes: {len(data)}")

if __name__ == "__main__":
    main()


[Episode    0] success_episode0_steps321 -> steps expected: 65 (0..320 by 5)
  -> Added episode: key='success_episode0_steps65', array shape=(65, 192)
[Episode    1] success_episode1_steps341 -> steps expected: 69 (0..340 by 5)
  -> Added episode: key='success_episode1_steps69', array shape=(69, 192)
[Episode    2] success_episode2_steps314 -> steps expected: 63 (0..313 by 5)
  -> Added episode: key='success_episode2_steps63', array shape=(63, 192)
[Episode    3] success_episode3_steps302 -> steps expected: 61 (0..301 by 5)
  -> Added episode: key='success_episode3_steps61', array shape=(61, 192)
[Episode    4] success_episode4_steps313 -> steps expected: 63 (0..312 by 5)
  -> Added episode: key='success_episode4_steps63', array shape=(63, 192)
[Episode    5] success_episode5_steps317 -> steps expected: 64 (0..316 by 5)
  -> Added episode: key='success_episode5_steps64', array shape=(64, 192)
[Episode    6] success_episode6_steps310 -> steps expected: 62 (0..309 by 5)
  -> Added episod

#실패 데이터 전처리
4가지 전처리 과정이 포함됩니다.

resent18<br>
resent18 + robotdata<br>
resnet18 -> pca<br>
resnet18 -> pca + robotdata<br>

In [27]:
"""
apply resent18 (for fail)

structure
data = {
    "fail{case_number}_episode{episode_number}_steps{episode_length//5+1}_noise{noise_step//5+1}":
        [episode_length//5+1, 512*3] #front, top, wrist
    }
"""
"""
apply resnet18 -> pca (for fail)

structure
data = {
    "fail{case_number}_episode{episode_number}_steps{episode_length//5+1}_noise{noise_step//5+1}":
        [episode_length//5+1, 64*3] #front, top, wrist
    }
"""

import os
import re
import pickle
import numpy as np
import torch
from PIL import Image
from torchvision import models, transforms
import re

def load_resnet_feature_extractor(device):
    # pretrained ResNet-18, 마지막 fc 층 Identity로 대체
    model = models.resnet18(pretrained=True)
    model.fc = torch.nn.Identity()
    model.eval()
    model.to(device)
    return model

def load_pca_model(view):
    pca_path = f'model/model_pca_{view}_view.pkl'
    with open(pca_path, 'rb') as f:
        return pickle.load(f)

def get_sorted_image_paths(dir_path, view):
    # view_view_#.png 에서 # 추출 후 오름차순 정렬
    pattern = re.compile(fr'{view}_view_(\d+)\.png')
    files = []
    for fname in os.listdir(dir_path):
        m = pattern.match(fname)
        if m:
            idx = int(m.group(1))
            files.append((idx, os.path.join(dir_path, fname)))
    files.sort(key=lambda x: x[0])
    return [path for _, path in files]

def process_episode(ep_dir, model, pca_models, device):
    # 1) robot_state.npz에서 EE pose 불러오기
    state = np.load(os.path.join(ep_dir, 'robot_state.npz'))
    if 'eepose' in state:
        ee_pose = state['eepose']           # shape (T,7)
    elif 'EE_pose' in state:
        ee_pose = state['EE_pose']
    else:
        raise KeyError(f'EE pose key not found in {ep_dir}/robot_state.npz')

    # 2) 이미지 전처리 트랜스폼
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406],
                             std= [0.229,0.224,0.225]),
    ])

    # 3) 각 view별로 ResNet → PCA
    latents = []
    with torch.no_grad():
        for view in ['front', 'top', 'wrist']:
            view_dir = os.path.join(ep_dir, f'{view}_view')
            img_paths = get_sorted_image_paths(view_dir, view)
            feats512 = []
            for img_path in img_paths:
                img = Image.open(img_path).convert('RGB')
                x = transform(img).unsqueeze(0).to(device)
                f512 = model(x).cpu().numpy().reshape(-1)  # (512,)
                feats512.append(f512)
            feats512 = np.stack(feats512, axis=0)           # (T,512)
            #feats64  = pca_models[view].transform(feats512) # (T,64)
            latents.append(feats512)

    # 4) front+top+wrist+ee_pose 합치기 → (T,199)
    data_episode = np.concatenate([latents[0], latents[1], latents[2]], axis=1)
    return data_episode

def main():
    root_dir = '/AILAB-summer-school-2025/fail_data_raw'
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # ResNet-18 & PCA 모델 로드
    model = load_resnet_feature_extractor(device)
    pca_models = {
        view: load_pca_model(view)
        for view in ['front', 'top', 'wrist']
    }

    processed = {}
    # 케이스 디렉토리 순회
    for case_name in sorted(os.listdir(root_dir)):
        case_dir = os.path.join(root_dir, case_name)
        if not os.path.isdir(case_dir):
            continue

        # 각 에피소드 디렉토리 순회
        for ep_name in sorted(os.listdir(case_dir)):
            ep_dir = os.path.join(case_dir, ep_name)
            if not os.path.isdir(ep_dir):
                continue

            # 정규표현식으로 정확히 fail{n}_episode{m}_step{s}_noise{v} 패턴만 매칭
            m = re.match(r'^(fail\d+)_episode(\d+)_step(\d+)_noise(\d+)$', ep_name)
            if not m:
                print(f'[Warning] unexpected folder name "{ep_name}", skipping.')
                continue

            fail_str       = m.group(1)             # ex) 'fail1'
            episode_num    = m.group(2)             # ex) '2'
            step_len       = int(m.group(3)) //5 +1
            noise_val      = int(m.group(4))        # ex) 180
            noise_idx      = noise_val // 5 + 1     # 5로 나눈 몫 + 1
            key            = f'{fail_str}_episode{episode_num}_step{step_len}_noise{noise_idx}'
            print(f'Processing {key} ...')
            processed[key] = process_episode(ep_dir, model, pca_models, device)

    # 결과 저장
    out_path = 'dataset/fail_data_resnet18_robotX.pkl'
    with open(out_path, 'wb') as f:
        pickle.dump(processed, f)
    print(f'Saved processed data to {out_path}')

if __name__ == '__main__':
    main()



Processing fail1_episode1_step70_noise37 ...
Processing fail1_episode2_step70_noise38 ...
Processing fail1_episode3_step70_noise35 ...
Processing fail1_episode4_step70_noise35 ...
Processing fail1_episode5_step70_noise35 ...
Processing fail2_episode1_step70_noise7 ...
Processing fail2_episode2_step70_noise7 ...
Processing fail2_episode3_step70_noise7 ...
Processing fail2_episode4_step70_noise7 ...
Processing fail2_episode5_step70_noise7 ...
Processing fail3_episode1_step70_noise41 ...
Processing fail3_episode2_step70_noise41 ...
Processing fail3_episode3_step70_noise41 ...
Processing fail3_episode4_step70_noise41 ...
Processing fail3_episode5_step70_noise41 ...
Processing fail4_episode1_step70_noise49 ...
Processing fail4_episode2_step70_noise46 ...
Processing fail4_episode3_step70_noise49 ...
Processing fail4_episode4_step70_noise50 ...
Processing fail4_episode5_step70_noise50 ...
Processing fail5_episode4_step70_noise1 ...
Processing fail5_episode5_step70_noise1 ...
[Warning] unexpec

In [ ]:
"""
apply resent18 + robotdata (for fail)

structure
data = {
    "fail{case_number}_episode{episode_number}_steps{episode_length//5+1}_noise{noise_step//5+1}":
        [episode_length//5+1, 512*3+7] #front, top, wrist, robot_data
    }
"""

import os
import re
import pickle
import numpy as np
import torch
from PIL import Image
from torchvision import models, transforms
import re

def load_resnet_feature_extractor(device):
    # pretrained ResNet-18, 마지막 fc 층 Identity로 대체
    model = models.resnet18(pretrained=True)
    model.fc = torch.nn.Identity()
    model.eval()
    model.to(device)
    return model

def load_pca_model(view):
    pca_path = f'model/model_pca_{view}_view.pkl'
    with open(pca_path, 'rb') as f:
        return pickle.load(f)

def get_sorted_image_paths(dir_path, view):
    # view_view_#.png 에서 # 추출 후 오름차순 정렬
    pattern = re.compile(fr'{view}_view_(\d+)\.png')
    files = []
    for fname in os.listdir(dir_path):
        m = pattern.match(fname)
        if m:
            idx = int(m.group(1))
            files.append((idx, os.path.join(dir_path, fname)))
    files.sort(key=lambda x: x[0])
    return [path for _, path in files]

def process_episode(ep_dir, model, pca_models, device):
    # 1) robot_state.npz에서 EE pose 불러오기
    state = np.load(os.path.join(ep_dir, 'robot_state.npz'))
    if 'eepose' in state:
        ee_pose = state['eepose']           # shape (T,7)
    elif 'EE_pose' in state:
        ee_pose = state['EE_pose']
    else:
        raise KeyError(f'EE pose key not found in {ep_dir}/robot_state.npz')

    # 2) 이미지 전처리 트랜스폼
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406],
                             std= [0.229,0.224,0.225]),
    ])

    # 3) 각 view별로 ResNet → PCA
    latents = []
    with torch.no_grad():
        for view in ['front', 'top', 'wrist']:
            view_dir = os.path.join(ep_dir, f'{view}_view')
            img_paths = get_sorted_image_paths(view_dir, view)
            feats512 = []
            for img_path in img_paths:
                img = Image.open(img_path).convert('RGB')
                x = transform(img).unsqueeze(0).to(device)
                f512 = model(x).cpu().numpy().reshape(-1)  # (512,)
                feats512.append(f512)
            feats512 = np.stack(feats512, axis=0)           # (T,512)
            #feats64  = pca_models[view].transform(feats512) # (T,64)
            latents.append(feats512)

    # 4) front+top+wrist+ee_pose 합치기 → (T,199)
    data_episode = np.concatenate([latents[0], latents[1], latents[2], ee_pose], axis=1)
    return data_episode

def main():
    root_dir = '/AILAB-summer-school-2025/fail_data_raw'
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # ResNet-18 & PCA 모델 로드
    model = load_resnet_feature_extractor(device)
    pca_models = {
        view: load_pca_model(view)
        for view in ['front', 'top', 'wrist']
    }

    processed = {}
    # 케이스 디렉토리 순회
    for case_name in sorted(os.listdir(root_dir)):
        case_dir = os.path.join(root_dir, case_name)
        if not os.path.isdir(case_dir):
            continue

        # 각 에피소드 디렉토리 순회
        for ep_name in sorted(os.listdir(case_dir)):
            ep_dir = os.path.join(case_dir, ep_name)
            if not os.path.isdir(ep_dir):
                continue

            # 정규표현식으로 정확히 fail{n}_episode{m}_step{s}_noise{v} 패턴만 매칭
            m = re.match(r'^(fail\d+)_episode(\d+)_step(\d+)_noise(\d+)$', ep_name)
            if not m:
                print(f'[Warning] unexpected folder name "{ep_name}", skipping.')
                continue

            fail_str       = m.group(1)             # ex) 'fail1'
            episode_num    = m.group(2)             # ex) '2'
            step_len       = int(m.group(3)) //5 +1
            noise_val      = int(m.group(4))        # ex) 180
            noise_idx      = noise_val // 5 + 1     # 5로 나눈 몫 + 1
            key            = f'{fail_str}_episode{episode_num}_step{step_len}_noise{noise_idx}'
            print(f'Processing {key} ...')
            processed[key] = process_episode(ep_dir, model, pca_models, device)

    # 결과 저장
    out_path = 'dataset/fail_data_resnet18_robotO.pkl'
    with open(out_path, 'wb') as f:
        pickle.dump(processed, f)
    print(f'Saved processed data to {out_path}')

if __name__ == '__main__':
    main()



In [25]:
"""
apply resnet18 -> pca (for fail)

structure
data = {
    "fail{case_number}_episode{episode_number}_steps{episode_length//5+1}_noise{noise_step//5+1}":
        [episode_length//5+1, 64*3] #front, top, wrist
    }
"""

import os
import re
import pickle
import numpy as np
import torch
from PIL import Image
from torchvision import models, transforms
import re

def load_resnet_feature_extractor(device):
    # pretrained ResNet-18, 마지막 fc 층 Identity로 대체
    model = models.resnet18(pretrained=True)
    model.fc = torch.nn.Identity()
    model.eval()
    model.to(device)
    return model

def load_pca_model(view):
    pca_path = f'model/model_pca_{view}_view.pkl'
    with open(pca_path, 'rb') as f:
        return pickle.load(f)

def get_sorted_image_paths(dir_path, view):
    # view_view_#.png 에서 # 추출 후 오름차순 정렬
    pattern = re.compile(fr'{view}_view_(\d+)\.png')
    files = []
    for fname in os.listdir(dir_path):
        m = pattern.match(fname)
        if m:
            idx = int(m.group(1))
            files.append((idx, os.path.join(dir_path, fname)))
    files.sort(key=lambda x: x[0])
    return [path for _, path in files]

def process_episode(ep_dir, model, pca_models, device):
    # 1) robot_state.npz에서 EE pose 불러오기
    state = np.load(os.path.join(ep_dir, 'robot_state.npz'))
    if 'eepose' in state:
        ee_pose = state['eepose']           # shape (T,7)
    elif 'EE_pose' in state:
        ee_pose = state['EE_pose']
    else:
        raise KeyError(f'EE pose key not found in {ep_dir}/robot_state.npz')

    # 2) 이미지 전처리 트랜스폼
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406],
                             std= [0.229,0.224,0.225]),
    ])

    # 3) 각 view별로 ResNet → PCA
    latents = []
    with torch.no_grad():
        for view in ['front', 'top', 'wrist']:
            view_dir = os.path.join(ep_dir, f'{view}_view')
            img_paths = get_sorted_image_paths(view_dir, view)
            feats512 = []
            for img_path in img_paths:
                img = Image.open(img_path).convert('RGB')
                x = transform(img).unsqueeze(0).to(device)
                f512 = model(x).cpu().numpy().reshape(-1)  # (512,)
                feats512.append(f512)
            feats512 = np.stack(feats512, axis=0)           # (T,512)
            feats64  = pca_models[view].transform(feats512) # (T,64)
            latents.append(feats64)

    # 4) front+top+wrist+ee_pose 합치기 → (T,199)
    data_episode = np.concatenate([latents[0], latents[1], latents[2]], axis=1)
    return data_episode

def main():
    root_dir = '/AILAB-summer-school-2025/fail_data_raw'
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # ResNet-18 & PCA 모델 로드
    model = load_resnet_feature_extractor(device)
    pca_models = {
        view: load_pca_model(view)
        for view in ['front', 'top', 'wrist']
    }

    processed = {}
    # 케이스 디렉토리 순회
    for case_name in sorted(os.listdir(root_dir)):
        case_dir = os.path.join(root_dir, case_name)
        if not os.path.isdir(case_dir):
            continue

        # 각 에피소드 디렉토리 순회
        for ep_name in sorted(os.listdir(case_dir)):
            ep_dir = os.path.join(case_dir, ep_name)
            if not os.path.isdir(ep_dir):
                continue

            # 정규표현식으로 정확히 fail{n}_episode{m}_step{s}_noise{v} 패턴만 매칭
            m = re.match(r'^(fail\d+)_episode(\d+)_step(\d+)_noise(\d+)$', ep_name)
            if not m:
                print(f'[Warning] unexpected folder name "{ep_name}", skipping.')
                continue

            fail_str       = m.group(1)             # ex) 'fail1'
            episode_num    = m.group(2)             # ex) '2'
            step_len       = int(m.group(3)) //5 +1
            noise_val      = int(m.group(4))        # ex) 180
            noise_idx      = noise_val // 5 + 1     # 5로 나눈 몫 + 1
            key            = f'{fail_str}_episode{episode_num}_step{step_len}_noise{noise_idx}'
            print(f'Processing {key} ...')
            processed[key] = process_episode(ep_dir, model, pca_models, device)

    # 결과 저장
    out_path = 'dataset/fail_data_resnet18_pca_robotX.pkl'
    with open(out_path, 'wb') as f:
        pickle.dump(processed, f)
    print(f'Saved processed data to {out_path}')

if __name__ == '__main__':
    main()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Processing fail1_episode1_step70_noise37 ...
Processing fail1_episode2_step70_noise38 ...
Processing fail1_episode3_step70_noise35 ...
Processing fail1_episode4_step70_noise35 ...
Processing fail1_episode5_step70_noise35 ...
Processing fail2_episode1_step70_noise7 ...
Processing fail2_episode2_step70_noise7 ...
Processing fail2_episode3_step70_noise7 ...
Processing fail2_episode4_step70_noise7 ...
Processing fail2_episode5_step70_noise7 ...
Processing fail3_episode1_step70_noise41 ...
Processing fail3_episode2_step70_noise41 ...
Processing fail3_episode3_step70_noise41 ...
Processing fail3_episode4_step70_noise41 ...
Processing fail3_episode5_step70_noise41 ...
Processing fail4_episode1_step70_noise49 ...
Processing fail4_episode2_step70_noise46 ...
Processing fail4_episode3_step70_noise49 ...
Processing fail4_episode4_step70_noise50 ...
Processing fail4_episode5_step70_noise50 ...
Processing fail5_episode4_step70_noise1 ...
Processing fail5_episode5_step70_noise1 ...
[Warning] unexpec

In [23]:
"""
apply resnet18 -> pca + robotdata (for fail)

structure
data = {
    "fail{case_number}_episode{episode_number}_steps{episode_length//5+1}_noise{noise_step//5+1}":
        [episode_length//5+1, 64*3+7] #front, top, wrist, robot_data
    }
"""

import os
import re
import pickle
import numpy as np
import torch
from PIL import Image
from torchvision import models, transforms
import re

def load_resnet_feature_extractor(device):
    # pretrained ResNet-18, 마지막 fc 층 Identity로 대체
    model = models.resnet18(pretrained=True)
    model.fc = torch.nn.Identity()
    model.eval()
    model.to(device)
    return model

def load_pca_model(view):
    pca_path = f'model/model_pca_{view}_view.pkl'
    with open(pca_path, 'rb') as f:
        return pickle.load(f)

def get_sorted_image_paths(dir_path, view):
    # view_view_#.png 에서 # 추출 후 오름차순 정렬
    pattern = re.compile(fr'{view}_view_(\d+)\.png')
    files = []
    for fname in os.listdir(dir_path):
        m = pattern.match(fname)
        if m:
            idx = int(m.group(1))
            files.append((idx, os.path.join(dir_path, fname)))
    files.sort(key=lambda x: x[0])
    return [path for _, path in files]

def process_episode(ep_dir, model, pca_models, device):
    # 1) robot_state.npz에서 EE pose 불러오기
    state = np.load(os.path.join(ep_dir, 'robot_state.npz'))
    if 'eepose' in state:
        ee_pose = state['eepose']           # shape (T,7)
    elif 'EE_pose' in state:
        ee_pose = state['EE_pose']
    else:
        raise KeyError(f'EE pose key not found in {ep_dir}/robot_state.npz')

    # 2) 이미지 전처리 트랜스폼
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406],
                             std= [0.229,0.224,0.225]),
    ])

    # 3) 각 view별로 ResNet → PCA
    latents = []
    with torch.no_grad():
        for view in ['front', 'top', 'wrist']:
            view_dir = os.path.join(ep_dir, f'{view}_view')
            img_paths = get_sorted_image_paths(view_dir, view)
            feats512 = []
            for img_path in img_paths:
                img = Image.open(img_path).convert('RGB')
                x = transform(img).unsqueeze(0).to(device)
                f512 = model(x).cpu().numpy().reshape(-1)  # (512,)
                feats512.append(f512)
            feats512 = np.stack(feats512, axis=0)           # (T,512)
            feats64  = pca_models[view].transform(feats512) # (T,64)
            latents.append(feats64)

    # 4) front+top+wrist+ee_pose 합치기 → (T,199)
    data_episode = np.concatenate([latents[0], latents[1], latents[2], ee_pose], axis=1)
    return data_episode

def main():
    root_dir = '/AILAB-summer-school-2025/fail_data_raw'
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # ResNet-18 & PCA 모델 로드
    model = load_resnet_feature_extractor(device)
    pca_models = {
        view: load_pca_model(view)
        for view in ['front', 'top', 'wrist']
    }

    processed = {}
    # 케이스 디렉토리 순회
    for case_name in sorted(os.listdir(root_dir)):
        case_dir = os.path.join(root_dir, case_name)
        if not os.path.isdir(case_dir):
            continue

        # 각 에피소드 디렉토리 순회
        for ep_name in sorted(os.listdir(case_dir)):
            ep_dir = os.path.join(case_dir, ep_name)
            if not os.path.isdir(ep_dir):
                continue

            # 정규표현식으로 정확히 fail{n}_episode{m}_step{s}_noise{v} 패턴만 매칭
            m = re.match(r'^(fail\d+)_episode(\d+)_step(\d+)_noise(\d+)$', ep_name)
            if not m:
                print(f'[Warning] unexpected folder name "{ep_name}", skipping.')
                continue

            fail_str       = m.group(1)             # ex) 'fail1'
            episode_num    = m.group(2)             # ex) '2'
            step_len       = int(m.group(3)) //5 +1
            noise_val      = int(m.group(4))        # ex) 180
            noise_idx      = noise_val // 5 + 1     # 5로 나눈 몫 + 1
            key            = f'{fail_str}_episode{episode_num}_step{step_len}_noise{noise_idx}'
            print(f'Processing {key} ...')
            processed[key] = process_episode(ep_dir, model, pca_models, device)

    # 결과 저장
    out_path = 'dataset/fail_data_resnet18_pca_robotO.pkl'
    with open(out_path, 'wb') as f:
        pickle.dump(processed, f)
    print(f'Saved processed data to {out_path}')

if __name__ == '__main__':
    main()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Processing fail1_episode1_step70_noise37 ...
Processing fail1_episode2_step70_noise38 ...
Processing fail1_episode3_step70_noise35 ...
Processing fail1_episode4_step70_noise35 ...
Processing fail1_episode5_step70_noise35 ...
Processing fail2_episode1_step70_noise7 ...
Processing fail2_episode2_step70_noise7 ...
Processing fail2_episode3_step70_noise7 ...
Processing fail2_episode4_step70_noise7 ...
Processing fail2_episode5_step70_noise7 ...
Processing fail3_episode1_step70_noise41 ...
Processing fail3_episode2_step70_noise41 ...
Processing fail3_episode3_step70_noise41 ...
Processing fail3_episode4_step70_noise41 ...
Processing fail3_episode5_step70_noise41 ...
Processing fail4_episode1_step70_noise49 ...
Processing fail4_episode2_step70_noise46 ...
Processing fail4_episode3_step70_noise49 ...
Processing fail4_episode4_step70_noise50 ...
Processing fail4_episode5_step70_noise50 ...
Processing fail5_episode4_step70_noise1 ...
Processing fail5_episode5_step70_noise1 ...
[Warning] unexpec